# Architecture du Modele de Segmentation

Ce notebook presente les architectures de modeles pour la segmentation semantique.

## Objectifs
- Implementer U-Net classique
- Implementer U-Net avec backbone pre-entraine (transfer learning)
- Definir les fonctions de perte (loss) adaptees
- Definir les metriques (IoU, Dice)
- Tester les modeles avec des donnees factices

## 1. Imports et Configuration

In [ ]:
import json
import numpy as np
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

# Seed pour reproductibilite
tf.random.set_seed(42)
np.random.seed(42)

TensorFlow version: 2.20.0
GPU disponible: []


In [2]:
# Charger la configuration
DATA_DIR = Path('../data')

with open(DATA_DIR / 'config.json', 'r') as f:
    config = json.load(f)

IMG_HEIGHT = config['img_height']
IMG_WIDTH = config['img_width']
N_CLASSES = config['n_classes']
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

print(f"Input shape: {INPUT_SHAPE}")
print(f"Nombre de classes: {N_CLASSES}")

Input shape: (256, 512, 3)
Nombre de classes: 8


## 2. Blocs de base pour U-Net

In [3]:
def conv_block(inputs, n_filters, kernel_size=3, batch_norm=True):
    """
    Bloc de convolution: Conv -> BN -> ReLU -> Conv -> BN -> ReLU
    
    Args:
        inputs: Tensor d'entree
        n_filters: Nombre de filtres
        kernel_size: Taille du kernel
        batch_norm: Utiliser batch normalization
    
    Returns:
        Tensor de sortie
    """
    x = layers.Conv2D(n_filters, kernel_size, padding='same', kernel_initializer='he_normal')(inputs)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(n_filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    return x


def encoder_block(inputs, n_filters, pool_size=(2, 2), dropout_rate=0.3):
    """
    Bloc encodeur: Conv Block -> MaxPool -> Dropout
    
    Returns:
        skip: Connection pour le decodeur
        pool: Sortie apres pooling
    """
    skip = conv_block(inputs, n_filters)
    pool = layers.MaxPooling2D(pool_size)(skip)
    pool = layers.Dropout(dropout_rate)(pool)
    return skip, pool


def decoder_block(inputs, skip, n_filters, kernel_size=3, dropout_rate=0.3):
    """
    Bloc decodeur: UpConv -> Concat(skip) -> Dropout -> Conv Block
    
    Args:
        inputs: Tensor d'entree (du niveau inferieur)
        skip: Skip connection (de l'encodeur)
        n_filters: Nombre de filtres
    
    Returns:
        Tensor de sortie
    """
    x = layers.Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(inputs)
    x = layers.Concatenate()([x, skip])
    x = layers.Dropout(dropout_rate)(x)
    x = conv_block(x, n_filters)
    return x

print("Blocs de base definis")

Blocs de base definis


## 3. U-Net Classique

In [4]:
def build_unet(input_shape, n_classes, filters=[64, 128, 256, 512, 1024]):
    """
    Construit un modele U-Net classique.
    
    Architecture:
    - Encodeur: 4 blocs de downsampling
    - Bottleneck: bloc central
    - Decodeur: 4 blocs de upsampling avec skip connections
    
    Args:
        input_shape: Shape des images d'entree (H, W, C)
        n_classes: Nombre de classes de sortie
        filters: Liste du nombre de filtres par niveau
    
    Returns:
        Modele Keras
    """
    inputs = layers.Input(shape=input_shape)
    
    # Encodeur
    skip1, pool1 = encoder_block(inputs, filters[0])
    skip2, pool2 = encoder_block(pool1, filters[1])
    skip3, pool3 = encoder_block(pool2, filters[2])
    skip4, pool4 = encoder_block(pool3, filters[3])
    
    # Bottleneck
    bottleneck = conv_block(pool4, filters[4])
    
    # Decodeur
    dec4 = decoder_block(bottleneck, skip4, filters[3])
    dec3 = decoder_block(dec4, skip3, filters[2])
    dec2 = decoder_block(dec3, skip2, filters[1])
    dec1 = decoder_block(dec2, skip1, filters[0])
    
    # Sortie
    outputs = layers.Conv2D(n_classes, (1, 1), activation='softmax')(dec1)
    
    model = Model(inputs, outputs, name='UNet')
    return model

# Creer le modele U-Net
unet_model = build_unet(INPUT_SHAPE, N_CLASSES)
unet_model.summary()

Model: "UNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 512,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 512,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 512,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 512,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 512,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 512,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 256,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128, 256,  │          0 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 256,  │     73,856 │ dropout[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 256,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 256,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 256,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 128,   │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64, 128,   │          0 │ max_pooling2d_1[

 Total params: 31,055,752 (118.47 MB)

 Trainable params: 31,043,976 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [5]:
# Version plus legere pour machines avec moins de memoire
def build_unet_light(input_shape, n_classes):
    """
    U-Net leger avec moins de filtres.
    Adapte pour l'entrainement sur CPU ou GPU avec peu de memoire.
    """
    return build_unet(input_shape, n_classes, filters=[32, 64, 128, 256, 512])

unet_light = build_unet_light(INPUT_SHAPE, N_CLASSES)
print(f"\nU-Net Light: {unet_light.count_params():,} parametres")
print(f"U-Net Standard: {unet_model.count_params():,} parametres")


U-Net Light: 7,772,104 parametres
U-Net Standard: 31,055,752 parametres


## 4. U-Net avec Backbone Pre-entraine (Transfer Learning)

In [6]:
def build_unet_vgg16(input_shape, n_classes, freeze_encoder=True):
    """
    U-Net avec encodeur VGG16 pre-entraine sur ImageNet.
    
    Le transfer learning permet d'obtenir de meilleurs resultats
    avec moins de donnees d'entrainement.
    
    Args:
        input_shape: Shape des images
        n_classes: Nombre de classes
        freeze_encoder: Geler les poids de l'encodeur
    
    Returns:
        Modele Keras
    """
    # Charger VGG16 sans les couches fully connected
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Geler les poids si demande
    if freeze_encoder:
        for layer in base_model.layers:
            layer.trainable = False
    
    # Recuperer les skip connections depuis VGG16
    # block1_conv2: 256x512x64
    # block2_conv2: 128x256x128
    # block3_conv3: 64x128x256
    # block4_conv3: 32x64x512
    # block5_conv3: 16x32x512 (bottleneck)
    
    skip1 = base_model.get_layer('block1_conv2').output  # 64 filters
    skip2 = base_model.get_layer('block2_conv2').output  # 128 filters
    skip3 = base_model.get_layer('block3_conv3').output  # 256 filters
    skip4 = base_model.get_layer('block4_conv3').output  # 512 filters
    
    # Bottleneck (sortie de VGG16)
    bottleneck = base_model.get_layer('block5_conv3').output  # 512 filters
    
    # Decodeur
    dec4 = decoder_block(bottleneck, skip4, 512)
    dec3 = decoder_block(dec4, skip3, 256)
    dec2 = decoder_block(dec3, skip2, 128)
    dec1 = decoder_block(dec2, skip1, 64)
    
    # Sortie
    outputs = layers.Conv2D(n_classes, (1, 1), activation='softmax')(dec1)
    
    model = Model(base_model.input, outputs, name='UNet_VGG16')
    return model

# Creer le modele
unet_vgg16 = build_unet_vgg16(INPUT_SHAPE, N_CLASSES, freeze_encoder=True)
print(f"U-Net VGG16: {unet_vgg16.count_params():,} parametres")
print(f"Parametres entrainables: {sum([tf.keras.backend.count_params(w) for w in unet_vgg16.trainable_weights]):,}")

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)

In [ ]:
def build_unet_mobilenet(input_shape, n_classes, freeze_encoder=True):
    """
    U-Net avec encodeur MobileNetV2 pre-entraine.
    
    MobileNetV2 est plus leger que VGG16, ideal pour le deploiement
    sur des systemes embarques (vehicules autonomes).
    """
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    
    if freeze_encoder:
        for layer in base_model.layers:
            layer.trainable = False
    
    # Skip connections MobileNetV2
    # Les noms des couches peuvent varier selon la version de Keras
    skip1 = base_model.get_layer('block_1_expand_relu').output   # 128x256
    skip2 = base_model.get_layer('block_3_expand_relu').output   # 64x128
    skip3 = base_model.get_layer('block_6_expand_relu').output   # 32x64
    skip4 = base_model.get_layer('block_13_expand_relu').output  # 16x32
    
    # Bottleneck
    bottleneck = base_model.get_layer('block_16_project').output  # 8x16
    
    # Decodeur
    x = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(bottleneck)
    x = layers.Concatenate()([x, skip4])
    x = conv_block(x, 256)
    
    x = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(x)
    x = layers.Concatenate()([x, skip3])
    x = conv_block(x, 128)
    
    x = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(x)
    x = layers.Concatenate()([x, skip2])
    x = conv_block(x, 64)
    
    x = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(x)
    x = layers.Concatenate()([x, skip1])
    x = conv_block(x, 32)
    
    # Upsampling final pour atteindre la taille d'origine
    x = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(x)
    x = conv_block(x, 16)
    
    outputs = layers.Conv2D(n_classes, (1, 1), activation='softmax')(x)
    
    model = Model(base_model.input, outputs, name='UNet_MobileNetV2')
    return model

# Test
try:
    unet_mobile = build_unet_mobilenet(INPUT_SHAPE, N_CLASSES)
    print(f"U-Net MobileNetV2: {unet_mobile.count_params():,} parametres")
except Exception as e:
    print(f"Erreur MobileNet: {e}")
    print("MobileNetV2 peut necessiter des ajustements selon la taille d'image")

/tmp/ipykernel_33153/1550562856.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)


U-Net MobileNetV2: 5,579,544 parametres


## 5. Fonctions de Perte (Loss Functions)

In [ ]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Calcule le coefficient de Dice.
    
    Dice = 2 * |A inter B| / (|A| + |B|)
    
    Valeur entre 0 et 1, ou 1 = prediction parfaite.
    """
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    """
    Dice Loss = 1 - Dice Coefficient
    
    Bonne pour les problemes de segmentation avec classes desequilibrees.
    """
    return 1 - dice_coefficient(y_true, y_pred)


def categorical_focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    """
    Focal Loss pour classification multi-classes.
    
    Reduit le poids des exemples faciles pour se concentrer
    sur les exemples difficiles.
    
    FL(p) = -alpha * (1-p)^gamma * log(p)
    """
    y_pred = tf.keras.backend.clip(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
    cross_entropy = -y_true * tf.keras.backend.log(y_pred)
    focal_weight = alpha * tf.keras.backend.pow(1 - y_pred, gamma)
    focal_loss = focal_weight * cross_entropy
    return tf.keras.backend.mean(tf.keras.backend.sum(focal_loss, axis=-1))


def combined_loss(y_true, y_pred):
    """
    Combinaison de Categorical Cross-Entropy et Dice Loss.
    
    Cette combinaison donne souvent de meilleurs resultats
    que chaque loss utilisee seule.
    """
    cce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return cce + dice


print("Fonctions de perte definies:")
print("  - dice_loss: pour classes desequilibrees")
print("  - categorical_focal_loss: focus sur exemples difficiles")
print("  - combined_loss: CCE + Dice")

Fonctions de perte definies:
  - dice_loss: pour classes desequilibrees
  - categorical_focal_loss: focus sur exemples difficiles
  - combined_loss: CCE + Dice


## 6. Metriques

In [ ]:
def iou_score(y_true, y_pred, smooth=1e-6):
    """
    Calcule l'Intersection over Union (IoU) / Jaccard Index.
    
    IoU = |A inter B| / |A union B|
    
    Metrique standard pour la segmentation semantique.
    """
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)


class MeanIoU(tf.keras.metrics.Metric):
    """
    Mean Intersection over Union pour segmentation multi-classes.
    
    Calcule l'IoU pour chaque classe puis fait la moyenne.
    """
    def __init__(self, num_classes, name='mean_iou', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.total_iou = self.add_weight(name='total_iou', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        # Convertir one-hot en labels
        y_true_labels = tf.argmax(y_true, axis=-1)
        y_pred_labels = tf.argmax(y_pred, axis=-1)
        
        # Calculer IoU par classe
        iou_per_class = []
        for c in range(self.num_classes):
            true_c = tf.cast(tf.equal(y_true_labels, c), tf.float32)
            pred_c = tf.cast(tf.equal(y_pred_labels, c), tf.float32)
            
            intersection = tf.reduce_sum(true_c * pred_c)
            union = tf.reduce_sum(true_c) + tf.reduce_sum(pred_c) - intersection
            
            # Eviter division par zero
            iou = tf.where(union > 0, intersection / union, 0.0)
            iou_per_class.append(iou)
        
        mean_iou = tf.reduce_mean(tf.stack(iou_per_class))
        self.total_iou.assign_add(mean_iou)
        self.count.assign_add(1.0)
    
    def result(self):
        return self.total_iou / self.count
    
    def reset_state(self):
        self.total_iou.assign(0.0)
        self.count.assign(0.0)


print("Metriques definies:")
print("  - iou_score: IoU simple")
print("  - MeanIoU: mIoU multi-classes")
print("  - dice_coefficient: Dice/F1 score")

Metriques definies:
  - iou_score: IoU simple
  - MeanIoU: mIoU multi-classes
  - dice_coefficient: Dice/F1 score


## 7. Compilation du Modele

In [ ]:
def compile_model(model, learning_rate=1e-4, loss='combined'):
    """
    Compile le modele avec l'optimiseur et les metriques.
    
    Args:
        model: Modele Keras
        learning_rate: Taux d'apprentissage
        loss: 'combined', 'dice', 'focal', ou 'cce'
    """
    # Choisir la fonction de perte
    if loss == 'combined':
        loss_fn = combined_loss
    elif loss == 'dice':
        loss_fn = dice_loss
    elif loss == 'focal':
        loss_fn = categorical_focal_loss
    else:
        loss_fn = 'categorical_crossentropy'
    
    # Optimiseur Adam avec learning rate
    optimizer = Adam(learning_rate=learning_rate)
    
    # Metriques
    metrics = [
        'accuracy',
        dice_coefficient,
        MeanIoU(num_classes=N_CLASSES)
    ]
    
    model.compile(
        optimizer=optimizer,
        loss=loss_fn,
        metrics=metrics
    )
    
    print(f"Modele compile avec:")
    print(f"  - Loss: {loss}")
    print(f"  - Learning rate: {learning_rate}")
    print(f"  - Metriques: accuracy, dice, mIoU")
    
    return model

# Compiler le modele U-Net leger
model = build_unet_light(INPUT_SHAPE, N_CLASSES)
model = compile_model(model, learning_rate=1e-4, loss='combined')

Modele compile avec:
  - Loss: combined
  - Learning rate: 0.0001
  - Metriques: accuracy, dice, mIoU


## 8. Test avec Donnees Factices

In [ ]:
# Creer des donnees factices pour tester le modele
batch_size = 4
dummy_images = np.random.rand(batch_size, IMG_HEIGHT, IMG_WIDTH, 3).astype(np.float32)
dummy_masks = np.random.randint(0, N_CLASSES, (batch_size, IMG_HEIGHT, IMG_WIDTH))
dummy_masks_onehot = np.eye(N_CLASSES)[dummy_masks].astype(np.float32)

print(f"Dummy images shape: {dummy_images.shape}")
print(f"Dummy masks shape: {dummy_masks_onehot.shape}")

Dummy images shape: (4, 256, 512, 3)
Dummy masks shape: (4, 256, 512, 8)


In [ ]:
# Test de prediction
predictions = model.predict(dummy_images, verbose=0)
print(f"Predictions shape: {predictions.shape}")
print(f"Predictions range: [{predictions.min():.4f}, {predictions.max():.4f}]")
print(f"Sum per pixel (should be ~1): {predictions[0, 0, 0, :].sum():.4f}")

2026-01-16 11:17:08.978280: I external/local_xla/xla/service/service.cc:163] XLA service 0x757980003510 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2026-01-16 11:17:08.978317: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2026-01-16 11:17:08.998781: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-01-16 11:17:09.193360: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701
2026-01-16 11:17:11.133502: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.33GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2026-01-16 11:17:11.548676: W ext

Predictions shape: (4, 256, 512, 8)
Predictions range: [0.0319, 0.3532]
Sum per pixel (should be ~1): 1.0000


I0000 00:00:1768558634.509211   33517 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [ ]:
# Test d'entrainement sur quelques iterations
print("Test d'entrainement (2 epochs sur donnees factices)...")
history = model.fit(
    dummy_images, 
    dummy_masks_onehot, 
    epochs=2, 
    batch_size=2,
    verbose=1
)
print("\nTest reussi!")

Test d'entrainement (2 epochs sur donnees factices)...
Epoch 1/2


2026-01-16 11:17:23.890494: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.20GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2026-01-16 11:17:24.246503: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.21GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2026-01-16 11:17:24.484246: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.31GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2026-01-16 11:17:24.655464: W external/local_xla/xla/ts

2/2 ━━━━━━━━━━━━━━━━━━━━ 25s 68ms/step - accuracy: 0.1250 - dice_coefficient: 0.1250 - loss: 3.2477 - mean_iou: 0.0544
Epoch 2/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.1248 - dice_coefficient: 0.1251 - loss: 3.2421 - mean_iou: 0.0544

Test reussi!


## 9. Visualisation de l'Architecture

In [ ]:
# Sauvegarder un schema de l'architecture
try:
    from tensorflow.keras.utils import plot_model
    
    plot_model(
        model, 
        to_file='../models/unet_architecture.png',
        show_shapes=True,
        show_layer_names=True,
        dpi=100
    )
    print("Schema sauvegarde dans models/unet_architecture.png")
except Exception as e:
    print(f"Impossible de generer le schema: {e}")
    print("Installez graphviz et pydot pour generer les schemas")

You must install pydot (`pip install pydot`) for `plot_model` to work.
Schema sauvegarde dans models/unet_architecture.png


In [ ]:
# Afficher un resume des couches
def print_model_summary(model):
    """Affiche un resume simplifie du modele."""
    print(f"\nModele: {model.name}")
    print(f"Input shape: {model.input_shape}")
    print(f"Output shape: {model.output_shape}")
    print(f"Total params: {model.count_params():,}")
    print(f"Trainable params: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")
    print(f"Non-trainable params: {sum([tf.keras.backend.count_params(w) for w in model.non_trainable_weights]):,}")
    
    print("\nCouches principales:")
    for layer in model.layers:
        if isinstance(layer, (layers.Conv2D, layers.Conv2DTranspose, layers.MaxPooling2D)):
            # Utiliser layer.output.shape au lieu de layer.output_shape
            output_shape = layer.output.shape[1:]
            print(f"  {layer.name}: {output_shape}")

print_model_summary(model)


Modele: UNet
Input shape: (None, 256, 512, 3)
Output shape: (None, 256, 512, 8)
Total params: 7,772,104
Trainable params: 7,766,216
Non-trainable params: 5,888

Couches principales:
  conv2d_58: (256, 512, 32)
  conv2d_59: (256, 512, 32)
  max_pooling2d_8: (128, 256, 32)
  conv2d_60: (128, 256, 64)
  conv2d_61: (128, 256, 64)
  max_pooling2d_9: (64, 128, 64)
  conv2d_62: (64, 128, 128)
  conv2d_63: (64, 128, 128)
  max_pooling2d_10: (32, 64, 128)
  conv2d_64: (32, 64, 256)
  conv2d_65: (32, 64, 256)
  max_pooling2d_11: (16, 32, 256)
  conv2d_66: (16, 32, 512)
  conv2d_67: (16, 32, 512)
  conv2d_transpose_17: (32, 64, 256)
  conv2d_68: (32, 64, 256)
  conv2d_69: (32, 64, 256)
  conv2d_transpose_18: (64, 128, 128)
  conv2d_70: (64, 128, 128)
  conv2d_71: (64, 128, 128)
  conv2d_transpose_19: (128, 256, 64)
  conv2d_72: (128, 256, 64)
  conv2d_73: (128, 256, 64)
  conv2d_transpose_20: (256, 512, 32)
  conv2d_74: (256, 512, 32)
  conv2d_75: (256, 512, 32)
  conv2d_76: (256, 512, 8)


## 10. Callbacks pour l'Entrainement

In [ ]:
def get_callbacks(model_name='unet', patience=10):
    """
    Retourne les callbacks pour l'entrainement.
    
    Args:
        model_name: Nom pour les fichiers sauvegardes
        patience: Patience pour early stopping
    
    Returns:
        Liste de callbacks
    """
    # Creer le dossier models s'il n'existe pas
    models_dir = Path('../models')
    models_dir.mkdir(exist_ok=True)
    
    callbacks = [
        # Sauvegarder le meilleur modele
        ModelCheckpoint(
            filepath=str(models_dir / f'{model_name}_best.keras'),
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            verbose=1
        ),
        
        # Arreter si pas d'amelioration
        EarlyStopping(
            monitor='val_loss',
            patience=patience,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Reduire le learning rate si plateau
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=patience // 2,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    return callbacks

# Creer le dossier models
Path('../models').mkdir(exist_ok=True)
print("Callbacks definis:")
print("  - ModelCheckpoint: sauvegarde le meilleur modele")
print("  - EarlyStopping: arrete si pas d'amelioration")
print("  - ReduceLROnPlateau: reduit le learning rate")

Callbacks definis:
  - ModelCheckpoint: sauvegarde le meilleur modele
  - EarlyStopping: arrete si pas d'amelioration
  - ReduceLROnPlateau: reduit le learning rate


## 11. Export du Module Models

In [ ]:
# Sauvegarder les fonctions dans un module Python
models_code = '''
"""Architectures de modeles pour la segmentation semantique."""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam


def conv_block(inputs, n_filters, kernel_size=3, batch_norm=True):
    """Bloc de convolution double."""
    x = layers.Conv2D(n_filters, kernel_size, padding=\'same\', kernel_initializer=\'he_normal\'')(inputs)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation(\'relu\'')(x)
    x = layers.Conv2D(n_filters, kernel_size, padding=\'same\', kernel_initializer=\'he_normal\'')(x)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation(\'relu\'')(x)
    return x


def encoder_block(inputs, n_filters, dropout_rate=0.3):
    """Bloc encodeur."""
    skip = conv_block(inputs, n_filters)
    pool = layers.MaxPooling2D((2, 2))(skip)
    pool = layers.Dropout(dropout_rate)(pool)
    return skip, pool


def decoder_block(inputs, skip, n_filters, dropout_rate=0.3):
    """Bloc decodeur."""
    x = layers.Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding=\'same\'')(inputs)
    x = layers.Concatenate()([x, skip])
    x = layers.Dropout(dropout_rate)(x)
    x = conv_block(x, n_filters)
    return x


def build_unet(input_shape, n_classes, filters=[32, 64, 128, 256, 512]):
    """Construit un modele U-Net."""
    inputs = layers.Input(shape=input_shape)
    
    skip1, pool1 = encoder_block(inputs, filters[0])
    skip2, pool2 = encoder_block(pool1, filters[1])
    skip3, pool3 = encoder_block(pool2, filters[2])
    skip4, pool4 = encoder_block(pool3, filters[3])
    
    bottleneck = conv_block(pool4, filters[4])
    
    dec4 = decoder_block(bottleneck, skip4, filters[3])
    dec3 = decoder_block(dec4, skip3, filters[2])
    dec2 = decoder_block(dec3, skip2, filters[1])
    dec1 = decoder_block(dec2, skip1, filters[0])
    
    outputs = layers.Conv2D(n_classes, (1, 1), activation=\'softmax\'')(dec1)
    
    return Model(inputs, outputs, name=\'UNet\')


def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """Coefficient de Dice."""
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    """Dice loss."""
    return 1 - dice_coefficient(y_true, y_pred)


def combined_loss(y_true, y_pred):
    """CCE + Dice loss."""
    cce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return cce + dice
'''

src_dir = Path('../src')
with open(src_dir / 'models.py', 'w') as f:
    f.write(models_code)

print("Module models.py sauvegarde dans src/")

Module models.py sauvegarde dans src/


In [ ]:
print("\n" + "="*60)
print("RESUME - ARCHITECTURES DE MODELES")
print("="*60)
print("\nModeles disponibles:")
print("  1. U-Net classique (build_unet)")
print("  2. U-Net leger (build_unet_light)")
print("  3. U-Net + VGG16 (build_unet_vgg16)")
print("  4. U-Net + MobileNetV2 (build_unet_mobilenet)")
print("\nFonctions de perte:")
print("  - combined_loss: CCE + Dice (recommande)")
print("  - dice_loss: pour classes desequilibrees")
print("  - categorical_focal_loss: focus sur exemples difficiles")
print("\nMetriques:")
print("  - accuracy")
print("  - dice_coefficient")
print("  - MeanIoU")
print("\nFichiers crees:")
print("  - src/models.py")
print("  - models/ (dossier pour sauvegarder les modeles)")


RESUME - ARCHITECTURES DE MODELES

Modeles disponibles:
  1. U-Net classique (build_unet)
  2. U-Net leger (build_unet_light)
  3. U-Net + VGG16 (build_unet_vgg16)
  4. U-Net + MobileNetV2 (build_unet_mobilenet)

Fonctions de perte:
  - combined_loss: CCE + Dice (recommande)
  - dice_loss: pour classes desequilibrees
  - categorical_focal_loss: focus sur exemples difficiles

Metriques:
  - accuracy
  - dice_coefficient
  - MeanIoU

Fichiers crees:
  - src/models.py
  - models/ (dossier pour sauvegarder les modeles)


## Prochaines etapes

1. **Notebook 04**: Entrainement complet du modele
2. **Notebook 05**: Evaluation et visualisation des resultats